# Measuring agreement

Notebook prepared by Dr. Marzieh Amini with Dr. Miodrag Bolic. It is based on the R program oximetry.R (http://www.utdallas.edu/~pankaj/agreement_book/ ) and 	oxygen saturation data file, which are again based on Chapter 4 of the book P. K. Choudhary, H. N. Nagaraja, 
Measuring agreement: Models, Methods and Applications, Wiley, 2017. 

Please download data from http://www.utdallas.edu/~pankaj/agreement_book/datasets/oxygen_saturation.txt and place it in the same folder as this notebook.

In [63]:
################################################
# Oxygen Saturation Data        #
################################################
import numpy as np
import scipy
from scipy import stats
from astroML.stats import fit_bivariate_normal
from astroML.stats.random import bivariate_normal


import numpy as np
import scipy.io as sio
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy as sp

import scipy.stats as st
import statsmodels.stats.api as sms
################################################
# Oxygen Saturation Data        #
################################################
oxy=pd.read_csv('oxygen_saturation.txt', sep='\t', header = 0)
# data frame with two columns --- second = method 1, first = method 2
z=oxy.as_matrix(columns=None)
x,y=z[:,1],z[:,0]
 

In [67]:
########################################################################
# Analysis based on bivariate normal model     #
########################################################################
(mu_nr, sigma1_nr,sigma2_nr, alpha_nr) = fit_bivariate_normal(x, y, robust=False)

np.random.seed(0)
xx, cov = bivariate_normal(mu_nr, sigma1_nr, sigma2_nr, alpha_nr, size=10000,
                          return_cov=True)


sigma1 = np.sqrt(cov[0, 0])
sigma2 = np.sqrt(cov[1, 1])
sigma12 = cov[0, 1]
corr=sigma12/(sigma1*sigma2)
Mu1=mu_nr[0]
Mu2=mu_nr[1]

###############################################################################
# Agreement Evaluation #
###############################################################################
## Computing CCC  #
###################
CCC=(2*(sigma12))/((Mu1-Mu2)**2+sigma1**2+sigma2**2)
D=np.abs(x-y)
TDI=np.percentile(D, (100*0.95))
print("######################")  
print("Similarity Evaluation") 
print("######################")  
print("mu1-mu2:",Mu1-Mu2) 
print("sigma^2/sigma1^2:",sigma2**2/sigma1**2) 
print("######################")  
print("Agreement Evaluation") 
print("######################")  
print("CCC",CCC) 
print("TDI",TDI) 

###############################################################################
# confidence intervals
###############################################################################



def mean_confidence_interval(data, confidence=0.95):
    a = 1.0*np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return (m-h, m+h)
print("######################")  
print("%95 Confidence interval:")
print("######################")
print(mean_confidence_interval(xx, confidence=0.95))


a = xx
mean_confidence_interval(a)
st.t.interval(0.95, len(a)-1, loc=np.mean(a), scale=st.sem(a))
sms.DescrStatsW(a).tconfint_mean()

###############################################################################
# after recalibration #
###############################################################################
difMu=Mu2-Mu1
y_recalib=y-difMu

(mu_nr2, sigma1_nr2,sigma2_nr2, alpha_nr2) = fit_bivariate_normal(x, y_recalib, robust=False)

np.random.seed(0)
xx2, cov2 = bivariate_normal(mu_nr2, sigma1_nr2, sigma2_nr2, alpha_nr2, size=1000,
                          return_cov=True)


sigma1_recalib = np.sqrt(cov2[0, 0])
sigma2_recalib= np.sqrt(cov2[1, 1])
sigma12_recalib= cov2[0, 1]
corr_recalib=sigma12_recalib/(sigma1_recalib*sigma2_recalib)
Mu1_recalib=mu_nr2[0]
Mu2_recalib=mu_nr2[1]

###############################################################################
# evaluation of agreement after recalibration #
###############################################################################
## Computing CCC  #
###################
CCC2=(2*(sigma12_recalib))/((Mu1_recalib-Mu2_recalib)**2+sigma1_recalib**2+sigma2_recalib**2)
D2=np.abs(x-y_recalib)
TDI2=np.percentile(D2, (100*0.95))

print("##################################################################")  
print("Agreement Evaluation after recalibration") 
print("##################################################################")  
print("CCC after recalibration",CCC2) 
print("TDI after recalibration",TDI2) 

######################
Similarity Evaluation
######################
mu1-mu2: 0.4125
sigma^2/sigma1^2: 1.0153945946
######################
Agreement Evaluation
######################
CCC 0.989259272935
TDI 2.2
######################
%95 Confidence interval:
######################
(array([ 89.21179104,  89.21061457]), array([ 89.54872445,  89.54990092]))
##################################################################
Agreement Evaluation after recalibration
##################################################################
CCC after recalibration 0.990379353531
TDI after recalibration 1.84625


In [68]:
################################################
# Analysis based on mixed-effects model       #
# Please note that this part does not work    #
################################################
  # model fitting
from rpy2 import robjects as ro
%load_ext rpy2.ipython
from rpy2.robjects.packages import importr
#%%R
#results<-bvn.fit(oxy)
md = smf.mixedlm("pos ~ osm", oxy, groups=oxy["osm"])
mdf = md.fit()
#mdl=md.loglike()
print(mdf.summary())
#print(mdl.summary())

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: pos      
No. Observations: 72      Method:             REML     
No. Groups:       60      Scale:              0.9306   
Min. group size:  1       Likelihood:         -119.1076
Max. group size:  3       Converged:          Yes      
Mean group size:  1.2                                  
-------------------------------------------------------
             Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept    -0.169    1.566 -0.108 0.914 -3.238  2.899
osm           0.998    0.017 57.210 0.000  0.963  1.032
groups RE     0.582    0.755                           

